In [70]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import re
import yfinance as yf
import pandas_market_calendars as mcal

In [71]:
df = pd.read_csv('Assets\CONVICTIONSUMMARY_270_opt3_best2(1).csv', sep='|')

In [72]:
df2 = pd.DataFrame()
df2 = df.loc[df['TYPE'] == 'MERGED']

Extract Time and Date

In [73]:
df2["Time"] = df2.apply(lambda row: re.split(" ",row.DATE)[0], axis=1)

df2["DATE"] = df2.apply(lambda row: re.split(" ",row.DATE)[-1],axis=1)

<ipython-input-73-0de3e1e305d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Time"] = df2.apply(lambda row: re.split(" ",row.DATE)[0], axis=1)
<ipython-input-73-0de3e1e305d2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["DATE"] = df2.apply(lambda row: re.split(" ",row.DATE)[-1],axis=1)


In [74]:
df2.dtypes

DATE             object
ID               object
STOCK            object
SECTOR           object
TYPE             object
CONVICTIONS     float64
Time             object
dtype: object

In [75]:
df2['DATE'] = pd.to_datetime(
                          df2['DATE'],
                          format='%Y-%m-%d')

<ipython-input-75-3660468bee21>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['DATE'] = pd.to_datetime(


In [76]:
df2 = df2.sort_values(by="DATE")

In [77]:
df2.dtypes

DATE            datetime64[ns]
ID                      object
STOCK                   object
SECTOR                  object
TYPE                    object
CONVICTIONS            float64
Time                    object
dtype: object

## Cleaning

In [78]:
df2.columns = ['DATE', 'ID', 'STOCK',"SECTOR","TYPE","SCORE","TIME"]

In [79]:
df2 = df2.loc[df2['SCORE'] != 0]

## Adding returns

### gettting info

In [80]:
list_of_companies = df2["STOCK"].unique()
list_of_dates = df2["DATE"].unique()

In [81]:
start_date = df2.DATE.min()
end_date = df2.DATE.max()

In [82]:
final_date = end_date + pd.DateOffset(months=12)
print(final_date)

2005-07-14 00:00:00


### Define functions

In [83]:
def get_history_data(ticker_symbol, start_date, end_date):
    tickerData = yf.Ticker(ticker_symbol)
    tickerDf = tickerData.history(period='1d', start=start_date, end=end_date)
    tickerDf["STOCK"] = ticker_symbol

    return tickerDf

    

In [84]:
def get_closest_trading_day(day,trading_days):
    trading_days = trading_days[trading_days>=pd.to_datetime(day, utc=True)]
    return trading_days.sort_values()[0]

In [85]:
def get_traning_days(start_date, end_date,stock_market_name):
    nyse = mcal.get_calendar(stock_market_name)
    valid_days = nyse.valid_days(start_date=start_date, end_date=final_date)
    return valid_days

In [86]:
def get_return_on_date(date,df):
  try:
    x = df.loc[date]
    x = float(x['Close'])
  except:
    x = 0.0
  return x

In [87]:
def calculate_returns(start_date, df, trading_days):
    returns = pd.DataFrame(columns=['DATE','STOCK','1MReturn','3MReturn','6MRerturn','12MReturn'])
    
    date1m = get_closest_trading_day(start_date + pd.DateOffset(months=1),trading_days)
    date3m = get_closest_trading_day(start_date + pd.DateOffset(months=3),trading_days)
    date6m = get_closest_trading_day(start_date + pd.DateOffset(months=6),trading_days)
    date12m = get_closest_trading_day(start_date + pd.DateOffset(months=12),trading_days)

    name = df['STOCK']
    try:
        current = df.loc[start_date]
    except:
        return returns
    data1m = get_return_on_date(date1m,df)
    data3m = get_return_on_date(date3m,df)
    data6m = get_return_on_date(date6m,df)
    data12m = get_return_on_date(date12m,df)

    current = float(current['Close'])

    return1m = (data1m - current)/current
    return3m = (data3m - current)/current
    return6m = (data6m - current)/current
    return12m = (data6m - current)/current


    returns = pd.DataFrame([[start_date,name[0],return1m, return3m, return6m,return12m]], columns=['DATE','STOCK','1MReturn',"3MReturn","6MReturn","12MReturn"])

 

    return returns

In [88]:
def calculate_returns_for_date(list_of_dates,df,trading_days):
    returns = pd.DataFrame(columns=['DATE','STOCK','1MReturn','3MReturn','6MReturn','12MReturn'])
    for d in list_of_dates:
        temp = calculate_returns(pd.to_datetime(d),df,trading_days)
        
        returns = returns.append(temp, ignore_index=True)
    
    return returns

In [89]:
def calculate_returns_for_companies(list_of_dates,list_of_companies,start_date, end_date):
    returns = pd.DataFrame(columns=['DATE','STOCK','1MReturn','3MReturn','6MReturn','12MReturn'])
    trading_days = get_traning_days(start_date,end_date, "NYSE")
    for c in list_of_companies:
        c = c.replace(".XX1","")
        c = c.replace(".XX2","")
        data = get_history_data(c,start_date, end_date)
        len_of_data = data.shape[0]
        if len_of_data !=0:
            returns_for_company = calculate_returns_for_date(list_of_dates,data,trading_days)
            returns = returns.append(returns_for_company, ignore_index=True)
        
    return returns
    

### Get Data

In [90]:
test = calculate_returns_for_companies(list_of_dates, list_of_companies[1:10],start_date, end_date)

- PKZ: No data found for this date range, symbol may be delisted
- PALDF: No data found, symbol may be delisted
- CHRS: Data doesn't exist for startDate = 1076454000, endDate = 1089756000
- AGU: No data found for this date range, symbol may be delisted
- BJS: No data found for this date range, symbol may be delisted


In [91]:
test.head()

,DATE,STOCK,1MReturn,3MReturn,6MReturn,12MReturn,6MRerturn
0,2004-02-11,TTEK,-0.071624,-0.238595,-1.0,-1.0,NaN
1,2004-02-18,TTEK,-0.040246,-0.233428,-1.0,-1.0,NaN
2,2004-02-25,TTEK,-0.017266,-0.203357,-1.0,-1.0,NaN
3,2004-03-03,TTEK,0.073336,-0.151044,-1.0,-1.0,NaN
4,2004-03-10,TTEK,-0.006816,-0.154820,-1.0,-1.0,NaN


### Aditional data

In [94]:
tickerData = yf.Ticker("SU")
tickerDf = tickerData.history(period='1d', start=start_date)

In [95]:
tickerDf

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2004-02-10,8.120847,8.370670,8.108197,8.332723,1641400,0.0,0.0
2004-02-11,8.411781,8.452891,8.304262,8.402294,1280800,0.0,0.0
2004-02-12,8.383320,8.383320,8.187256,8.196743,950400,0.0,0.0
2004-02-13,8.301099,8.405456,8.073412,8.114522,1313400,0.0,0.0
2004-02-17,8.285290,8.291614,8.149310,8.212556,1207000,0.0,0.0
...,...,...,...,...,...,...,...
2022-04-01,32.439999,33.090000,32.410000,32.770000,4133300,0.0,0.0
2022-04-04,33.160000,33.480000,32.849998,33.430000,6159900,0.0,0.0
2022-04-05,33.630001,34.209999,32.950001,32.990002,5589300,0.0,0.0
